In [ ]:
import json
import os
import re
import random
from datetime import datetime

DATA_FILE = 'accounts.json'

# ---------------------------
# Data handling functions
# ---------------------------
def load_data():
    if os.path.exists(DATA_FILE):
        with open(DATA_FILE, 'r') as file:
            try:
                return json.load(file)
            except json.JSONDecodeError:
                return {}
    return {}

def save_data(data):
    with open(DATA_FILE, 'w') as file:
        json.dump(data, file, indent=4)

# ---------------------------
# Password strength checker
# ---------------------------
def is_strong_password(password):
    return (
        len(password) >= 8 and
        re.search(r'[A-Z]', password) and
        re.search(r'[a-z]', password) and
        re.search(r'[0-9]', password) and
        re.search(r'[!@#$%^&*(),.?":{}|<>]', password)
    )

# ---------------------------
# Generate unique account number
# ---------------------------
def generate_account_number(existing_numbers):
    while True:
        acc_num = str(random.randint(1000000000, 9999999999))
        if acc_num not in existing_numbers:
            return acc_num

# ---------------------------
# Banking App Functions
# ---------------------------
def register(data):
    username = input("Enter Username: ").strip()
    password = input("Enter Password: ").strip()

    if " " in username:
        print("❌ Username must not contain spaces.")
        return
    if username in data:
        print("❌ Username already exists.")
        return
    if not is_strong_password(password):
        print("⚠️ Password must have uppercase, lowercase, number & special character.")
        return
    if username == "" or password == "":
        print("⚠️ Username and password are required.")
        return

    existing_numbers = [
        details['account_number']
        for details in data.values()
        if isinstance(details, dict) and 'account_number' in details
    ]
    account_number = generate_account_number(existing_numbers)

    data[username] = {
        'password': password,
        'account_number': account_number,
        'balance': 0.0,
        'transactions': []
    }
    save_data(data)
    print(f"✅ Account created successfully! Your Account Number: {account_number}")

def login(data):
    username = input("Username: ").strip()
    password = input("Password: ").strip()

    if username in data and data[username]['password'] == password:
        print(f"✅ Welcome back, {username}!")
        return username
    else:
        print("❌ Invalid credentials.")
        return None

def deposit(user, data):
    try:
        amount = float(input("Enter amount to deposit: "))
        if amount > 0:
            data[user]['balance'] += amount
            data[user]['transactions'].append({"type": "Deposit", "amount": amount, "time": str(datetime.now())})
            save_data(data)
            print(f"✅ Deposited {amount:.2f} successfully!")
        else:
            print("⚠️ Enter a valid amount.")
    except ValueError:
        print("⚠️ Invalid number entered.")

def withdraw(user, data):
    try:
        amount = float(input("Enter amount to withdraw: "))
        if amount > 0 and amount <= data[user]['balance']:
            data[user]['balance'] -= amount
            data[user]['transactions'].append({"type": "Withdraw", "amount": amount, "time": str(datetime.now())})
            save_data(data)
            print(f"✅ Withdrawn {amount:.2f} successfully!")
        elif amount > data[user]['balance']:
            print("❌ Insufficient funds.")
        else:
            print("⚠️ Enter a valid amount.")
    except ValueError:
        print("⚠️ Invalid number entered.")

def transfer(user, data):
    recipient_acc = input("Recipient Account Number: ").strip()
    try:
        amount = float(input("Enter amount to transfer: "))
        if amount <= 0:
            print("⚠️ Enter a valid amount.")
            return
        if amount > data[user]['balance']:
            print("❌ Insufficient funds.")
            return

        recipient_user = None
        for uname, details in data.items():
            if details.get('account_number') == recipient_acc:
                recipient_user = uname
                break

        if recipient_user is None:
            print("❌ Recipient account not found.")
        elif recipient_user == user:
            print("⚠️ You cannot transfer money to yourself.")
        else:
            data[user]['balance'] -= amount
            data[user]['transactions'].append({
                "type": "Transfer Sent",
                "to": recipient_acc,
                "amount": amount,
                "time": str(datetime.now())
            })

            data[recipient_user]['balance'] += amount
            data[recipient_user]['transactions'].append({
                "type": "Transfer Received",
                "from": data[user]['account_number'],
                "amount": amount,
                "time": str(datetime.now())
            })

            save_data(data)
            print(f"✅ Transferred {amount:.2f} to account {recipient_acc} successfully!")
    except ValueError:
        print("⚠️ Invalid number entered.")

def check_balance(user, data):
    print(f"💵 Current Balance: {data[user]['balance']:.2f}")
    show_txn = input("Show transaction history? (y/n): ").lower()
    if show_txn == 'y':
        if data[user]['transactions']:
            for txn in data[user]['transactions']:
                if txn["type"] in ["Deposit", "Withdraw"]:
                    print(f"{txn['time']} - {txn['type']}: {txn['amount']:.2f}")
                elif txn["type"] == "Transfer Sent":
                    print(f"{txn['time']} - Sent {txn['amount']:.2f} to {txn['to']}")
                elif txn["type"] == "Transfer Received":
                    print(f"{txn['time']} - Received {txn['amount']:.2f} from {txn['from']}")
        else:
            print("No transactions yet.")

# ---------------------------
# Main Program Loop
# ---------------------------
def main():
    data = load_data()
    logged_in_user = None

    while True:
        if not logged_in_user:
            print("\n--- Banking App ---")
            print("1. Register")
            print("2. Login")
            print("3. Exit")
            choice = input("Choose an option: ")

            if choice == "1":
                register(data)
            elif choice == "2":
                logged_in_user = login(data)
            elif choice == "3":
                print("Goodbye!")
                break
            else:
                print("Invalid choice.")
        else:
            print(f"\n--- Welcome {logged_in_user} ---")
            print("1. Deposit")
            print("2. Withdraw")
            print("3. Transfer Money")
            print("4. Check Balance")
            print("5. Logout")
            choice = input("Choose an option: ")

            if choice == "1":
                deposit(logged_in_user, data)
            elif choice == "2":
                withdraw(logged_in_user, data)
            elif choice == "3":
                transfer(logged_in_user, data)
            elif choice == "4":
                check_balance(logged_in_user, data)
            elif choice == "5":
                logged_in_user = None
            else:
                print("Invalid choice.")

if __name__ == "__main__":
    main()



--- Banking App ---
1. Register
2. Login
3. Exit
